In [1]:
# Import Modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#modeling tools
import statsmodels.api as sm

import lightgbm as lgb

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix, auc, roc_auc_score, roc_curve

%matplotlib inline
plt.style.use('dark_background')

%load_ext autoreload
%autoreload 2

pd.set_option('display.min_row', 25)
pd.set_option('display.max_column', 100)
pd.set_option('display.max_colwidth', 300)

In [16]:
feat_eng = pd.read_csv('../data/processed/feateng_collisions.csv', low_memory=False)

In [17]:
feat_eng.columns

Index(['Datetime', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INJURIES',
       'SERIOUSINJURIES', 'FATALITIES', 'INCDATE', 'JUNCTIONTYPE',
       'SDOT_COLCODE', 'SDOT_COLDESC', 'INATTENTIONIND', 'UNDERINFL',
       'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT', 'SDOTCOLNUM',
       'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY',
       'HITPARKEDCAR', 'latlon_tup', 'bikerack_dist', 'radar_dist',
       't_circles_dist', 'marked_cross_dist'],
      dtype='object')

In [25]:
new_features = feat_eng[['OBJECTID', 'INTKEY',  'latlon_tup', 'bikerack_dist', 'radar_dist',
       't_circles_dist', 'marked_cross_dist']]

In [5]:
collisions = pd.read_pickle('../data/processed/cleaned_data.pkl')
collisions.head()

,X,Y,OBJECTID,INCKEY,ADDRTYPE,INTKEY,LOCATION,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,JUNCTIONTYPE,SDOT_COLCODE,UNDERINFL,ST_COLCODE,CROSSWALKKEY,SPEEDING_Y,INATTENTIONIND_Y,HITPARKEDCAR_Y,PEDROWNOTGRNT_Y,WEATHER_Adverse,WEATHER_Good,WEATHER_Unknown,ROADCOND_Adverse,ROADCOND_Dry,ROADCOND_Unknown,LIGHTCOND_Dark,LIGHTCOND_Daylight,LIGHTCOND_Unknown,LIGHTCOND_VeryDark,SEVERITYCODE_Injury,SEVERITYCODE_PropertyDamage,SEVERITYCODE_Unknown
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01-01,NaN,NaN,11627,25040,Block,NaN,BATTERY ST TUNNEL NB BETWEEN ALASKAN WY VI NB AND AURORA AVE N,2,0,0,2,1,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),14.0,0,13,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0
2004-01-01,-122.318760,47.604359,12525,24635,Block,NaN,E ALDER ST BETWEEN 10TH AVE AND 11TH AVE,2,0,0,2,0,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),16.0,0,32,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0
2004-01-01,-122.320780,47.614076,14172,26463,Intersection,29745.0,BROADWAY AND E PIKE ST,4,0,0,2,1,0,0,2004/01/01 00:00:00+00,At Intersection (intersection related),11.0,0,28,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0
2004-01-01,-122.383351,47.529183,6538,19530,Block,NaN,SW ROSE ST BETWEEN 39TH AVE SW AND 41ST AVE SW,2,0,0,2,0,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),14.0,0,32,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0
2004-01-01,-122.329974,47.708637,9665,22520,Block,NaN,N NORTHGATE WAY BETWEEN CORLISS AVE N AND 1ST AVE NE,1,0,0,1,0,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),28.0,0,50,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0


In [27]:
collisions.reset_index(inplace=True)

In [31]:
new_collisions = collisions.merge(new_features, how='left', on='OBJECTID')

In [32]:
new_collisions.head()

,Datetime,X,Y,OBJECTID,INCKEY,ADDRTYPE,INTKEY_x,LOCATION,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,JUNCTIONTYPE,SDOT_COLCODE,UNDERINFL,ST_COLCODE,CROSSWALKKEY,SPEEDING_Y,INATTENTIONIND_Y,HITPARKEDCAR_Y,PEDROWNOTGRNT_Y,WEATHER_Adverse,WEATHER_Good,WEATHER_Unknown,ROADCOND_Adverse,ROADCOND_Dry,ROADCOND_Unknown,LIGHTCOND_Dark,LIGHTCOND_Daylight,LIGHTCOND_Unknown,LIGHTCOND_VeryDark,SEVERITYCODE_Injury,SEVERITYCODE_PropertyDamage,SEVERITYCODE_Unknown,INTKEY_y,latlon_tup,bikerack_dist,radar_dist,t_circles_dist,marked_cross_dist
0,2004-01-01,NaN,NaN,11627,25040,Block,NaN,BATTERY ST TUNNEL NB BETWEEN ALASKAN WY VI NB AND AURORA AVE N,2,0,0,2,1,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),14.0,0,13,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,NaN,"(47.687805803319904, -122.37545723173001)",0.062183,0.500988,0.063781,0.083897
1,2004-01-01,-122.318760,47.604359,12525,24635,Block,NaN,E ALDER ST BETWEEN 10TH AVE AND 11TH AVE,2,0,0,2,0,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),16.0,0,32,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,NaN,"(47.7191338070879, -122.29531896585699)",0.015159,0.408913,0.137101,0.001891
2,2004-01-01,-122.320780,47.614076,14172,26463,Intersection,29745.0,BROADWAY AND E PIKE ST,4,0,0,2,1,0,0,2004/01/01 00:00:00+00,At Intersection (intersection related),11.0,0,28,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,37176.0,"(47.6920510510355, -122.360659037339)",0.096524,0.257162,0.126424,0.092283
3,2004-01-01,-122.383351,47.529183,6538,19530,Block,NaN,SW ROSE ST BETWEEN 39TH AVE SW AND 41ST AVE SW,2,0,0,2,0,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),14.0,0,32,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,NaN,"(47.7196056381102, -122.35621791966899)",0.289216,0.346730,0.156457,0.024229
4,2004-01-01,-122.329974,47.708637,9665,22520,Block,NaN,N NORTHGATE WAY BETWEEN CORLISS AVE N AND 1ST AVE NE,1,0,0,1,0,0,0,2004/01/01 00:00:00+00,Mid-Block (not related to intersection),28.0,0,50,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,NaN,"(47.6768825661541, -122.36250907509199)",0.059942,0.443346,0.061665,0.063178


In [33]:
# save to csv and pickle
new_collisions.to_csv('../data/processed/collisions_feateng.csv')
new_collisions.to_pickle('../data/processed/collisions_feateng.pkl')